In [5]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder

In [6]:
df = pd.read_csv('../data/all_matches.csv')
df.replace('2007/08','2008',inplace=True)
df.replace('2009/10','2010',inplace=True)
df.replace('2020/21','2020',inplace=True)
df['season'] = df['season'].astype(int)
df = df.loc[df['season'] > 2017]
df = df.loc[df['wides']!=1]
df = df.loc[df['ball']<6.1]
columns_to_remove = [ 'season', 'start_date', 'venue', 'batting_team', 'bowling_team', 'non_striker', 'striker', 'extras','byes','legbyes','penalty','other_wicket_type','player_dismissed','other_player_dismissed']
df.drop(labels=columns_to_remove, axis=1, inplace=True)
# print(df)
df = df.fillna(0)
# df.to_csv('ball.csv')
# print(df['wicket_type'].unique())
print(df.columns)


C:\Users\samyak jain\AppData\Local\Programs\Python\Python38-32\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
Index(['match_id', 'innings', 'ball', 'bowler', 'runs_off_bat', 'wides',
       'noballs', 'wicket_type'],
      dtype='object')


In [7]:
diff_balls = df['bowler'].unique()
main = pd.DataFrame(columns= ['bowler','balls_bowled','runs_conceeded','wickets'])

for diff in diff_balls:
    _df = df.loc[df['bowler']==diff]
    playerName = diff[0] + " " +  diff.split(" ")[-1][0:]
    
    runs_bat = _df['runs_off_bat'].sum().astype(int)
    totalwides = _df['wides'].sum().astype(int)
    totalnoballs = _df['noballs'].sum().astype(int)
    totalruns = runs_bat + totalwides + totalnoballs 
    
    dfBall = _df.loc[(_df['wides']!=1) & (_df['noballs']!=1) ]
    totalballs = len(dfBall)

    dfWickets = _df.loc[(_df['wicket_type']!= 0) & (_df['wicket_type']!= 'run out') & (_df['wicket_type']!= 'retired hurt') ]
    wickets = len(dfWickets)

    X = [[playerName,totalballs,totalruns,wickets]]
    X1 = pd.DataFrame(X, columns=['bowler','balls_bowled','runs_conceeded','wickets'])
    main = pd.concat([main,X1])
    
main.to_csv('bowler2.csv', index=False)

In [8]:
dfBowl = pd.read_csv('bowler2.csv')

dfBowl['Economy'] = (dfBowl['runs_conceeded']/dfBowl['balls_bowled'])*6

a = dfBowl['Economy']
b = dfBowl['balls_bowled']
c = dfBowl['wickets']

rating = []

for i in range(len(dfBowl)):
    if ((a[i]<=7.50) & (b[i]>=240) & (c[i]>=10)):
        rating.append(9.50)
    elif ((a[i]<=8.00) & (b[i]>=120) & (c[i]>=20)):
        rating.append(9.25)
    elif ((a[i]<=8.00) & (c[i]>=5)):
        rating.append(9.00)
    elif ((a[i]<=7.50) & (b[i]>=150)):
        rating.append(8.75)
    elif ((a[i]<=8.0) & (b[i]>=150)):
        rating.append(8.50)
    elif ((a[i]<=8.50) &  (b[i]>=150)):
        rating.append(8.25)
    elif ((a[i]<=8.00)  &  (b[i]>=150) & (c[i]>=3)):
        rating.append(8.25)
    elif ((a[i]<=9.00) &  (b[i]>=150)):
        rating.append(8.00)
    elif ((a[i]<=9.50) &  (b[i]>=150)):
        rating.append(7.50)
    elif ((a[i]<=8.70) &  (b[i]>=90) & (c[i]>=5)):
        rating.append(8.25)
    elif ((a[i]<=9.0) & (c[i]>=5)):
        rating.append(7.25)
    elif ((a[i]<=9.0) &  (b[i]>=150) &(c[i]>=5)):
        rating.append(7.00)
    else:
        rating.append(8.00)
dfBowl['Rating'] = rating
dfBowl.to_csv('bowler2.csv')